## Cassandra:

Data Model Creation:
* Design a data model for an e-commerce platform to handle products, orders, and user information.
* Define appropriate column families and primary keys to ensure efficient querying.

In [78]:
from cassandra.cluster import Cluster
import uuid
from decimal import Decimal
from datetime import datetime

In [79]:
uuid.uuid4()

UUID('b2fb3c93-9d88-4e73-9a82-912f6cdc440e')

In [80]:
cluster = Cluster(["127.0.0.1"],port=9042)
session = cluster.connect()

In [84]:
session.execute("""CREATE KEYSPACE IF NOT EXISTS e_commerce
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1}""")

In [85]:
session.set_keyspace('e_commerce')

In [86]:
session.execute("""CREATE TABLE IF NOT EXISTS users (
        user_id UUID PRIMARY KEY,
        username TEXT,
        email TEXT)""")

session.execute("""CREATE TABLE IF NOT EXISTS products (
        product_id UUID PRIMARY KEY,
        product_name TEXT,
        price DECIMAL)""")

session.execute("""CREATE TABLE IF NOT EXISTS user_orders (
        user_id UUID,
        order_id UUID,
        order_date TIMESTAMP,
        product_name TEXT,
        quantity INT,
        total_amount DECIMAL,
        PRIMARY KEY (user_id, order_id))""")

Data Insertion and Retrieval:
* Insert sample data into the Cassandra database, including user information and product details.
* Retrieve a user's order history using CQL (Cassandra Query Language).

In [87]:
users_data = [
    (uuid.uuid4(), 'user1', 'user1@example.com'),
    (uuid.uuid4(), 'user2', 'user2@example.com'),
    (uuid.uuid4(), 'user3', 'user3@example.com')]

for user_data in users_data:
    session.execute(
        """INSERT INTO users (user_id, username, email)
        VALUES (%s, %s, %s)""",
        user_data)

products_data = [
    (uuid.uuid4(), 'Product A', Decimal('2')),
    (uuid.uuid4(), 'Product B', Decimal('2.5')),
    (uuid.uuid4(), 'Product C', Decimal('2.99'))]

for product_data in products_data:
    session.execute(
        """INSERT INTO products (product_id, product_name, price)
        VALUES (%s, %s, %s)""",
        product_data)

user_orders = [
    (uuid.uuid4(), users_data[0][0], datetime.now(), 'Product A', 1, Decimal('2')),
    (uuid.uuid4(), users_data[0][0], datetime.now(), 'Product C', 2, Decimal('5.98'))]

for order_data in user_orders:
    session.execute("""INSERT INTO user_orders (order_id, user_id, order_date, product_name, quantity, total_amount)
        VALUES (%s, %s, %s, %s, %s, %s)""",
        order_data)

In [88]:
user_id_to_retrieve = users_data[0][0]

result = session.execute(
    """SELECT * FROM user_orders WHERE user_id = %s""",
    (user_id_to_retrieve,))

for row in result:
    print(row.order_id, row.order_date, row.product_name, row.total_amount)

65277254-7289-41e4-bab0-33040908bf06 2023-08-11 20:22:57.712000 Product A 2
a96f856e-a98f-4e86-8245-5324f930c09f 2023-08-11 20:22:57.712000 Product C 5.98
